[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/train-text-to-image-tpu-tutorial/blob/main/prepare_dataset_colab.ipynb)

`create test.txt and copy 🍝 this html inside`
```html
<img src="https://cdn.pixabay.com/photo/2016/11/29/11/58/animal-1869339_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2017/05/15/09/59/cat-2314325_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2017/05/22/07/40/cat-2333413_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2018/11/09/18/52/cat-3805250_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2015/10/05/14/05/cat-972655_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2018/03/04/22/37/cat-3199463_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2017/08/16/20/16/yellow-cat-2649033_960_720.jpg" alt="cat">
<img src="https://cdn.pixabay.com/photo/2018/12/11/09/31/lovely-3868695_960_720.jpg" alt="cat">
```



In [ ]:
!pip install datasets bs4

In [ ]:
from huggingface_hub import notebook_login
!git config --global credential.helper store
notebook_login()

In [ ]:
!mkdir test

In [ ]:
import urllib.request, requests
from bs4 import BeautifulSoup

with open('/content/test.txt') as html:
    content = html.read()

soup = BeautifulSoup(content)
for imgtag in soup.find_all('img'):
  url=imgtag['src']
  name = url.split('/')[-1]
  headers={'user-agent': 'Mozilla/5.0'}
  r=requests.get(url, headers=headers)
  with open(f"/content/test/{name}", 'wb') as f:
      f.write(r.content)

In [ ]:
from datasets import load_dataset, Dataset, Image

with open('/content/test.txt') as html:
  content = html.read()

texts = []
images = []
soup = BeautifulSoup(content)
for imgtag in soup.find_all('img'):
  texts.append(imgtag['alt'])
  images.append(f"/content/test/{imgtag['src'].split('/')[-1]}")
  
ds = Dataset.from_dict({"image": images, "text": texts})
ds = ds.cast_column("image", Image())

In [ ]:
ds.push_to_hub("camenduru/test")

In [ ]:
# for unzipped images
import os
directory = '/content/images'
image_list = []
for subdir in os.listdir(directory):
    subdir_path = os.path.join(directory, subdir)
    if os.path.isdir(subdir_path):
        for filename in os.listdir(subdir_path):
            if filename.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_list.append(f'<img src="/content/images/{subdir}/{filename}" alt="{subdir}">')
output_file = 'image_list.txt'
with open(output_file, 'w') as f:
    for image_tag in image_list:
        f.write(image_tag + '\n')